In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, callback
import dash_bootstrap_components as dbc

In [14]:
df = pd.read_csv("lacrime6.csv")

In [7]:
#change datatypes for graphing purposes
df["hour"] = df["hour"].astype(int)
df["year"] = df["year"].astype(str)
df["month"] = df["month"].astype(int)

In [4]:
crime_rank = df.groupby(by=["year", "AREA NAME"])["Crm Cd Desc"].count().groupby("year").rank(method = "min", ascending = False).astype(int).to_frame().reset_index()

In [5]:
crime_rank = crime_rank.rename(columns = {"Crm Cd Desc" : "year_crime_rank"})

In [ ]:
#create background color for tiles based on crime rank
def crime_freq(row):
    if row["year_crime_rank"] <= 7:
        return "MidnightBlue"
    elif row["year_crime_rank"] > 7 and row["year_crime_rank"] < 15:
        return "RoyalBlue"
    else:
        return "DeepSkyBlue"

df["Frequency"] = df.apply (lambda row: crime_freq(row), axis = 1)

In [7]:
df = pd.merge(df, crime_rank, how = "inner", on = ["AREA NAME", "year"])
df.drop(columns = "Unnamed: 0", axis = 1, inplace = True)


In [15]:
df["DATE OCC"] = df["DATE OCC"].astype("datetime64[ns]")
df["year"] = df["DATE OCC"].dt.year
df["month"] = df["DATE OCC"].dt.month
df["day"] =  df["DATE OCC"].dt.day

In [15]:
external_stylesheets = [dbc.themes.BOOTSTRAP]

app = Dash(__name__, external_stylesheets = external_stylesheets)



app.layout = html.Div([
    dbc.Row([
        html.Label('Area Name:'),
        dcc.Dropdown(df["AREA NAME"].sort_values().unique(),
            id='area_search',
            value="77th Street",
            clearable=False
        ),
    ]),
    
    html.Br(),
    html.H2(id = "area_name", style={'textAlign' : "center"}),
    dbc.Row(
    [
    dbc.Col(id = '2020_rank', width = 3),
    dbc.Col(id = "2021_rank", width = 3),
    dbc.Col(id = "2022_rank", width = 3),
    dbc.Col(id = "2023_rank", width = 3)
    ]),
    html.Br(),
    dbc.Row([
    dbc.Col([dcc.Graph(id="year_month_graph")])
    ]),          
    dbc.Col([dcc.Graph(id="hour_graph")]),
    dbc.Row([
    dbc.Col([dcc.Graph(id="age_graph")], width = 6),
    dbc.Col([dcc.Graph(id = "sex_graph")], width = 6)
    ]),
    dbc.Col([dcc.Graph(id="age_sex_graph")]),
    dbc.Row([
    dbc.Col([dcc.Graph(id="race_graph")], width = 6),
    dbc.Col([dcc.Graph(id = "race_sex_graph")], width = 6)
    ]),
    dbc.Col([dcc.Graph(id="race_age_graph")]),
    dbc.Row([
    dbc.Col([dcc.Graph(id="top_ten_crimes")], width = 6),
    dbc.Col([dcc.Graph(id = "top_ten_sex")], width = 6)
    ]),
    
])

@app.callback(
    Output("2020_rank", "children"),
    Input("area_search", "value"))
def first_rank(area):
    data = df[df["AREA NAME"] == area]
    data = data[data["year"] == 2020]
    card1 = data["year_crime_rank"].reset_index()
    card1 = card1["year_crime_rank"][0]
    text_color = data["Frequency"].to_frame().reset_index()
    text_color = text_color["Frequency"][0]
    card_content = [
    dbc.CardBody([html.H5("2020 Crime Rank: " + str(card1))], style={'textAlign': 'center'})]

  
    return  dbc.Card(card_content, color = text_color, inverse=True)

@app.callback(
    Output("2021_rank", "children"),
    Input("area_search", "value"))
def second_rank(area):
    data = df[df["AREA NAME"] == area]
    data = data[data["year"] == 2021]
    card1 = data["year_crime_rank"].reset_index()
    card1 = card1["year_crime_rank"][0]
    text_color = data["Frequency"].to_frame().reset_index()
    text_color = text_color["Frequency"][0]
    card_content = [
    dbc.CardBody([html.H5("2021 Crime Rank: " + str(card1))], style={'textAlign': 'center'})]

  
    return  dbc.Card(card_content, color = text_color, inverse=True)

@app.callback(
    Output("2022_rank", "children"),
    Input("area_search", "value"))
def third_rank(area):
    data = df[df["AREA NAME"] == area]
    data = data[data["year"] == 2022]
    card1 = data["year_crime_rank"].reset_index()
    card1 = card1["year_crime_rank"][0]
    text_color = data["Frequency"].to_frame().reset_index()
    text_color = text_color["Frequency"][0]
    card_content = [
    dbc.CardBody([html.H5("2022 Crime Rank: " + str(card1))], style={'textAlign': 'center'})]

  
    return  dbc.Card(card_content, color = text_color, inverse=True)
@app.callback(
    Output("2023_rank", "children"),
    Input("area_search", "value"))
def fourth_rank(area):
    data = df[df["AREA NAME"] == area]
    data = data[data["year"] == 2023]
    card1 = data["year_crime_rank"].reset_index()
    card1 = card1["year_crime_rank"][0]
    text_color = data["Frequency"].to_frame().reset_index()
    text_color = text_color["Frequency"][0]
    card_content = [
    dbc.CardBody([html.H5("2023 Crime Rank: " + str(card1))], style={'textAlign': 'center'})]

  
    return  dbc.Card(card_content, color = text_color, inverse=True)

@app.callback(
    Output("year_month_graph", "figure"),
    Input("area_search", "value"))
def year_month(area):
    data = df[df["AREA NAME"] == area]
    df_years = data.groupby(by =["year", "month"])["Date Rptd"].count().to_frame().reset_index()
    df_years = df_years.rename(columns = {"Date Rptd" : "Number of Crimes"})
    fig = px.line(df_years, x = "month", y = "Number of Crimes" , color = "year", title = "Total Crime By Year and Month",
                 color_discrete_sequence=["DarkTurquoise", "DodgerBlue", "MidnightBlue", "Aquamarine"])
    fig.update_traces(hovertemplate = "<b>Month<b>: %{x}<br>"
                                      "<b>Crime Count<b>: %{y}<br>"
                                      "<extra></extra>", line_width = 4)
    fig.update_layout(
        xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
        ), 
        plot_bgcolor='white'
    )
    return fig
@app.callback(
    Output("hour_graph", "figure"),
    Input("area_search", "value"))
def by_hour(area):
    df_hour = df.groupby(by="hour")["Date Rptd"].count().to_frame().reset_index()
    df_hour = df_hour.rename(columns = {"Date Rptd" : "Number of Crimes"})
    fig = px.line(df_hour, x = "hour", y = "Number of Crimes", title = "Total Crime by Hour")
    fig.update_traces(line_color =  'MidnightBlue', line_width = 4, hovertemplate = "<b>Hour<b>: %{x}<br>"
                                                                                         "<b>Crime Count<b>: %{y}<br>"
                                                                                         "<extra></extra>")
    fig.update_layout(
        xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
        ),
        plot_bgcolor='white'
    )
    return fig
@app.callback(
    Output("age_graph", "figure"),
    Input("area_search", "value"))
def by_age(area):
    data = df[df["AREA NAME"] == area]
    df_age = data.groupby(by="age_category")["Date Rptd"].count().to_frame().reset_index()
    df_age = df_age.rename(columns = {"Date Rptd" : "Number of Crimes", "age_category" : "Age Category"})
    fig = px.bar(df_age, x = "Age Category", y = "Number of Crimes", text_auto = True, title = "Total Crime by Victim Age")
    fig.update_traces(marker_color = "MidnightBlue", marker_line_color =  'black', marker_line_width = 1.5, hovertemplate = "<b>Age Range<b>: %{x}<br>"
                                                                                         "<b>Crime Count<b>: %{y}<br>"
                                                                                         "<extra></extra>")
    fig.update_layout(plot_bgcolor='white')
    return fig
@app.callback(
    Output("sex_graph", "figure"),
    Input("area_search", "value"))
def by_sex(area):
    data = df[df["AREA NAME"] == area]
    df_sex = data.groupby(by="Vict Sex")["Date Rptd"].count().to_frame().reset_index()
    df_sex = df_sex.rename(columns = {"Date Rptd" : "Number of Crimes"})
    fig = px.bar(df_sex, x = "Vict Sex", y = "Number of Crimes", text_auto = True, title = "Total Crime by Victim Sex")
    fig.update_traces(marker_color = "MidnightBlue", marker_line_color =  'black', marker_line_width = 1.5, hovertemplate = "<b>Sex<b>: %{x}<br>"
                                                                                          "<b>Crime Count<b>: %{y}<br>"
                                                                                          "<extra></extra>")
    fig.update_layout(plot_bgcolor='white')
    return fig
@app.callback(
    Output("age_sex_graph", "figure"),
    Input("area_search", "value"))
def by_age_sex(area):
    data = df[df["AREA NAME"] == area]
    df_allage2 = df.groupby(by=["Vict Age", "Vict Sex"])["Date Rptd"].count().to_frame().reset_index()
    df_allage2 = df_allage2.rename(columns = {"Date Rptd" : "Number of Crimes"})
    df_allage2 = (df_allage2[df_allage2["Vict Age"] > 0])
    sexes = ["Male", "Female"]
    df_allage2 = df_allage2[df_allage2["Vict Sex"].isin(sexes)]
    fig = px.line(df_allage2, x = "Vict Age", y = "Number of Crimes", title = "Total Crime by Victim Age & Sex", color = "Vict Sex",
                 color_discrete_sequence=["MidnightBlue", "DodgerBlue"])
    fig.update_traces(hovertemplate = "<b>Victim Age<b>: %{x}<br>"
                                  "<b>Crime Count<b>: %{y}<br>"
                                  "<extra></extra>",
                     line_width = 4)
    fig.update_layout(
        xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
        ), plot_bgcolor='white')
    return fig
@app.callback(
    Output("race_graph", "figure"),
    Input("area_search", "value"))
def by_race(area):
    data = df[df["AREA NAME"] == area]
    df_race = data.groupby(by="Vict Descent")["Date Rptd"].count().to_frame().reset_index()
    df_race = df_race.rename(columns = {"Vict Descent" : "Race/Ethnicity", "Date Rptd" : "Number of Crimes"})
    races = ["Asian", "Black", "Hispanic/Latino", "White", "Other"]
    df_race = df_race[df_race["Race/Ethnicity"].isin(races)]
    fig = px.bar(df_race, x = "Race/Ethnicity", y = "Number of Crimes", title = "Total Crime by Victim Race", text_auto = True)
    fig.update_traces(marker_color = "MidnightBlue", marker_line_color =  'black', marker_line_width = 1.5)
    fig.update_layout(plot_bgcolor='white')
    return fig
@app.callback(
    Output("race_sex_graph", "figure"),
    Input("area_search", "value"))
def by_race_sex(area):
    data = df[df["AREA NAME"] == area]
    races = ["Asian", "Black", "Hispanic/Latino", "White", "Other"]
    sexes = ["Male", "Female"]
    df_race_gender = data.groupby(by=["Vict Sex", "Vict Descent"])["Date Rptd"].count().to_frame().reset_index()
    df_race_gender = df_race_gender.rename(columns = {"Vict Descent" : "Race/Ethnicity", "Date Rptd" : "Number of Crimes"})
    df_race_gender = df_race_gender[df_race_gender["Vict Sex"].isin(sexes)]
    df_race_gender = df_race_gender[df_race_gender["Race/Ethnicity"].isin(races)]
    fig = px.bar(df_race_gender, x = "Race/Ethnicity", y = "Number of Crimes", color = "Vict Sex", barmode = "group", title = "Total Crime by Victim Race & Gender", text_auto = True,
                 color_discrete_sequence=["MidnightBlue", "DodgerBlue"])
    fig.update_traces(marker_line_color =  'black', marker_line_width = 1.5)
    fig.update_layout(plot_bgcolor='white')
    return fig
@app.callback(
    Output("race_age_graph", "figure"),
    Input("area_search", "value"))
def by_race_age(area):
    data = df[df["AREA NAME"] == area]
    df_allage3 = data.groupby(by=["Vict Age", "Vict Descent"])["Date Rptd"].count().to_frame().reset_index()
    df_allage3 = df_allage3.rename(columns = {"Date Rptd" : "Number of Crimes", "Vict Descent" : "Race/Ethnicity"})
    df_allage3 = (df_allage3[df_allage3["Vict Age"] > 0])
    races = ["Asian", "Black", "Hispanic/Latino", "White", "Other"]
    df_allage3 = df_allage3[df_allage3["Race/Ethnicity"].isin(races)]
    fig = px.line(df_allage3, x = "Vict Age", y = "Number of Crimes", color = "Race/Ethnicity", title = "Total Crime by Victim Race & Age",
                 color_discrete_sequence=["DarkTurquoise", "DodgerBlue", "MidnightBlue", "Aquamarine", "Blue"])
    fig.update_layout(
        xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
        ), plot_bgcolor='white')
    fig.update_traces(line_width = 4)
    return fig
@app.callback(
    Output("top_ten_crimes", "figure"),
    Input("area_search", "value"))
def top_ten_area(area):
    data = df.groupby(by="AREA NAME")["Crm Cd Desc"].value_counts().to_frame().reset_index()
    data = data[data["AREA NAME"] == area]
    data = data.head(5)
    data = data.rename(columns = {"Crm Cd Desc" : "Crime Description", "count" : "Count"})
    data = data.drop(columns = "AREA NAME", axis = 1)
    rowOddColor = 'white'
    rowEvenColor = 'WhiteSmoke'
    fig = go.Figure(data=[go.Table(
            header=dict(values=list(data.columns),
                    fill_color='MidnightBlue',
                    line_color='black',
                    font=dict(color='white', size=14)),
            cells=dict(values=[data["Crime Description"], data["Count"]],
                   line_color='black',
                   fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor, rowEvenColor]*10],
                   align='left'))])
    fig.update_layout(title_text = "Top 5 Crimes")
    return fig
@app.callback(
    Output("top_ten_sex", "figure"),
    Input("area_search", "value"))
def top_ten_sex(area):
    data = df.groupby(by=["AREA NAME", "Vict Sex"])["Crm Cd Desc"].value_counts().to_frame().reset_index()
    data = data[data["AREA NAME"] == area]
    data = data[data["Vict Sex"] == "Female"]
    data = data.head(5)
    data = data.rename(columns = {"Crm Cd Desc" : "Crime Description", "count" : "Count"})
    data = data.drop(columns = ["AREA NAME", "Vict Sex"], axis = 1)
    rowOddColor = 'white'
    rowEvenColor = 'WhiteSmoke'
    fig = go.Figure(data=[go.Table(
            header=dict(values=list(data.columns),
                    fill_color='MidnightBlue',
                    line_color='black',
                    font=dict(color='white', size=14)),
            cells=dict(values=[data["Crime Description"], data["Count"]],
                   line_color='black',
                   fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor, rowEvenColor]*10],
                   align='left'))])
    fig.update_layout(title_text = "Top 5 Crimes Against Women")
    
    return fig

@app.callback(
    Output("area_name", "children"),
    Input("area_search", "value"))
def get_area_name(area):
    area_name = area + " Area Crime from 2020-2023"

    return area_name
if __name__ == "__main__":
    app.run()